# Felhantering

> Errors should never pass silently. Unless explicitly silenced.

Tim Peters, *The Zen of Python*

## Syntaxfel och *exceptions*

### Syntaxfel

Fel som uppstår när koden är skriven på ett sätt som Python-tolkaren inte kan tolka.


In [ ]:
print('this string is unterminated)

In [ ]:
for i in ['forgot', 'the', ':']
    print('so this is a syntax error')

### *Exceptions*

Fel som uppstår när koden är korrekt rent syntaxmässigt, men ändå inte kan exekveras.

#### `NameError`

In [ ]:
print(undefined_variable)  # pyright: ignore[reportUndefinedVariable]

#### `AttributeError`

In [ ]:
a = 42
a.upper()  # pyright: ignore[reportAttributeAccessIssue]

#### `ValueError`

In [ ]:
int('fifty three')

#### `TypeError`

In [ ]:
1 + '2'  # pyright: ignore[reportOperatorIssue]

#### `IndexError`

In [ ]:
[2, 4, 6][4]

`builtins` har alla inbyggda *exceptions*.

In [ ]:
[err for err in __builtins__.__dict__.keys() if 'Error' in err]

## `try ... except ... else`

### LBYL vs. EAFP

#### **LBYL**: *Look Before You Leap*
Försöker förutspå vad som kan gå fel och bygger logiska system med många `if ... else`-satser.

#### **EAFP**: *Easier to Ask for Forgiveness than Permission*
Förutsätter att allt är som det ska och hanterar *exceptions* med `try ... except` när de uppstår.

In [ ]:
a = 42
b = '23'

print(a + b)  # pyright: ignore[reportOperatorIssue]

print('This is also important!')

Vi kan tysta *exceptions* med `pass`.

In [ ]:
a = 42
b = '23'

try:
    print(a + b)  # pyright: ignore[reportOperatorIssue]
except:
    pass  # Explicitly silenced

print('This is also important!')

Det viktiga är att vi vet vad vi håller på med. Oftast kommer en tystad *exception* tillbaka och biter oss senare.

In [ ]:
a = 42
b = '23'

try:
    s = a + b  # pyright: ignore[reportOperatorIssue]
except:
    pass  # Explicitly silenced


In [ ]:
print(s)  # Variable is never defined

In [ ]:
a = 42
b = '23'

try:
    print(a + b)  # pyright: ignore[reportOperatorIssue]
except TypeError:
    print(int(a) + int(b))

In [ ]:
a = 42
b = 'tjugotre'

try:
    print(a + b)  # pyright: ignore[reportOperatorIssue]
except TypeError:
    print(int(a) + int(b))

In [ ]:
a = 42
b = 'tjugotre'

try:
    print(a + b)  # pyright: ignore[reportOperatorIssue]
except TypeError:
    try:
        print(int(a) + int(b))
    except ValueError as e:
        print(e)
        print('How do we handle this situation?')

## Lyfta *exceptions* med `raise`

Vi kan använda `isinstance` för att kolla om objektet vi hanterar är av en viss datatyp.

In [ ]:
a = 42
if not isinstance(a, str):
    raise TypeError('Object must be of type str')
else:
    print(a.upper())

In [ ]:
a = 'forty-two'
if not isinstance(a, str):
    raise TypeError('Object must be of type str')
else:
    print(a.upper())

`hasattr` returnerar `True` om ett attribut finns på ett visst objekt. Vi ändrar typen av *exception* jämfört med exemplena ovan.

In [ ]:
a = 'forty-two'
if not hasattr(a, 'upper'):
    raise AttributeError("Object must have attribute 'upper'")
else:
    print(a.upper())

In [ ]:
b = 42
if not hasattr(b, 'upper'):
    raise AttributeError("Object must have attribute 'upper'")
else:
    print(a.upper())

### Datatvätt med LBYL & EAFP



Vi läser in ett dataset av lite sämre kvalitet.^[Jag kan ha råkat ha sönder lite data från SMHI. 🤷] Det är en lista av dictionaries med nycklarna `date` och `temp`.

`date`-värdena beskriver datum i två olika format.

En del `temp`-värden är  `None`, och dessutom är det blandat mellan `.` och `,` som decimalavgränsare.

Vi ska här nedan se på två sätt att tvätta datan för att på sikt kunna göra ett linjediagram över temperaturerna.

In [ ]:
import json
import pandas as pd

temps = json.load(open('data/temps.json'))
temps

#### `date`

In [ ]:
# Eftersom vi vet att det är två olika format lämpar sig LBYL bättre

import datetime

for row in temps:
    if '-' in row['date']:  # Format YYYY-MM-DD
        y, m, d = [int(x) for x in row['date'].split('-')]
        new_date = datetime.date(y, m, d)
    elif '/' in row['date']:  # Format MM/DD/YY
        m, d, y = [int(x) for x in row['date'].split('/')]
        new_date = datetime.date(2000 + y, m, d)  # Lägg till 2000 till året för att få 2024
    else:
        raise Exception(f'Could not parse date {row["date"]}')  # Fånga eventuella undantag
    row['date'] = new_date

In [ ]:
temps

#### `temp`

In [ ]:
# Här är det bättre med EAFP eftersom vi har flera olika saker som kan gå fel

for row in temps:
    try:
        new_temp = float(row['temp'])
    except TypeError:  # Om värdet är None får vi ett TypeError
        new_temp = pd.NA
    except ValueError:  # Om decimalavgränsaren är ett , får vi ett ValueError
        new_temp = float(row['temp'].replace(',', '.'))
    except Exception as e:  # Fånga eventuella andra exceptions, lyft dem just nu
        raise e
    row['temp'] = new_temp

In [ ]:
temps

In [ ]:
df = pd.DataFrame(data=temps)
df = df.dropna(axis=0)
df['temp'] = df.temp.astype(float)
df.plot(kind='line', x='date', y='temp', figsize=(8, 4))

Sen ska vi inte glömma att Pandas är helt underbart och att det vi gjort ovan också kan göras med fyra rader kod:

In [ ]:
df2 = pd.read_json('data/temps.json')
df2['temp'] = df2.temp.str.replace(',', '.').astype(float)
df2.dropna(inplace=True)
df2.plot(kind='line', x= 'date', y='temp', figsize=(8, 4))

### Datatvätt med Pandas

Pandas kan automatiskt ändra de olika datumformaten till ett enhetligt format och göra om dem till sin egen `datetime64`-datatyp.

In [ ]:
df3 = pd.read_json('data/temps.json')
df3.date

`temp`-kolumnen är av `object`-datatypen och behöver lite hjälp att omvandlas till `float`.

In [ ]:
df3.temp

Vi kan komma åt `str`-metoder på värdena i en kolumn genom att ange `.str` efter kolumnens namn. Då kan vi köra `replace()` på `temp`-kolumnen och omvandla värdena till `float`. Pandas hanterar automatisk `None`-värden.

In [ ]:
df3.temp.str.replace(',', '.').astype(float)

Vi skriver över värdena i `temp`-kolumnen med de nya.

In [ ]:
df3['temp'] = df3.temp.str.replace(',', '.').astype(float)

In [ ]:
df3.temp

Nu kan vi droppa raderna med saknade värden.

In [ ]:
df3.dropna(inplace=True)


In [ ]:
df3

In [ ]:
df3.plot(kind='line', x= 'date', y='temp', figsize=(8, 4))